In [ ]:
!pip install lightly

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/Kaggle/data.zip' -d '/content/'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from cv2 import imread, createCLAHE # read and equalize images
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
from __future__ import print_function, division

# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# image imports
from skimage import io, transform
from PIL import Image

# general imports
import os
import time
from shutil import copyfile
from shutil import rmtree

# data science imports
import pandas as pd
import numpy as np
import csv

In [ ]:
from itertools import chain
import random
np.random.seed(25)

df = pd.read_csv('/content/Data_Entry_2017.csv')
df.drop(['OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'], axis=1, inplace=True)
num_obs = len(df)
# print('Number of observations:',num_obs)

my_glob = glob('/content/images*/images/*.png')
# print('Number of Observations: ', len(my_glob))

full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

train_val_list = pd.read_csv('/content/train_val_list.txt', header=None, names = ['image_list'])
test_list = pd.read_csv('/content/test_list.txt', header=None, names = ['image_list'])

train = df[df['Image Index'].isin(train_val_list['image_list'].values)].reset_index(drop=True)
test = df[df['Image Index'].isin(test_list['image_list'].values)].reset_index(drop=True)

labels_discard = ['Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Hernia', 'Pleural_Thickening']
for i in labels_discard:
    train = train[~train['Finding Labels'].str.contains(i)]
    test = test[~test['Finding Labels'].str.contains(i)]

# train = pd.concat([train[~train['Finding Labels'].str.contains('No Finding')],
#                   train[train['Finding Labels'].str.contains('No Finding')].drop_duplicates(subset=['Finding Labels', 'Patient ID', 'View Position'], keep='first')]).sort_values(by='Image Index')

def one_hot_enc(df):
    df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
    all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    for c_label in all_labels:
        if len(c_label)>1: # leave out empty labels
            df[c_label] = df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
    return df

train = one_hot_enc(train)
test = one_hot_enc(test)

train_image_paths = list(flatten(train['full_path'].values))
test_image_paths = list(flatten(test['full_path'].values))

l = np.unique(train['Patient ID'].values)
np.random.shuffle(l)
cut = int(np.round(((90/100)*len(l)), decimals=0))
train_values = l[:cut]
val_values = l[cut:]
train_dict = dict.fromkeys(train_values, 'train')
train_dict.update(dict.fromkeys(val_values, 'val'))
train['fold'] = train['Patient ID'].map(train_dict.get)

train = train[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
               'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
test['fold'] = 'test'
test = test[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
             'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
from PIL import Image

class CreateDataset(Dataset):

    def __init__(self, train, fold, transform=None):
        self.df = train
        self.transform = transform
        self.df = self.df[self.df['fold'] == fold]

        self.df = self.df.set_index("Image Index")
        self.PRED_LABEL = ['Atelectasis',
                           'Cardiomegaly',
                           'Effusion',
                           'Infiltration',
                           'Mass',
                           'Nodule',
                           'Pneumonia',
                           'Pneumothorax']
        RESULT_PATH = "results/"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = Image.open(self.df.iloc[idx, 0])
        image = image.convert('RGB')

        label = np.zeros(len(self.PRED_LABEL), dtype=int)
        for i in range(0, len(self.PRED_LABEL)):
            if(self.df[self.PRED_LABEL[i].strip()].iloc[idx].astype('int') > 0):
                label[i] = self.df[self.PRED_LABEL[i].strip()
                                  ].iloc[idx].astype('int')

#         labels = self.df.iloc[idx, 2:]
# #         landmarks = np.array([landmarks])
#         labels = torch.Tensor(labels)
# #         landmarks = landmarks.astype('float').reshape(-1, 2)

        if self.transform:
            image = self.transform(image)
#         sample = {'image': image, 'landmarks': landmarks}

        return image, label

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
N_LABELS = 8  # we are predicting 14 labels
data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomHorizontalFlip(),
        transforms.Resize(224),
        # because resize doesn't always give 224 x 224, this ensures 224 x
        # 224
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

transformed_datasets = {}
transformed_datasets['train'] = CreateDataset(train, 'train', transform=data_transforms['train'])
transformed_datasets['val'] = CreateDataset(train, 'val', transform=data_transforms['val'])

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(
    transformed_datasets['train'],
    batch_size=32,
    shuffle=True,
    num_workers=8,
    pin_memory=True)
dataloaders['val'] = torch.utils.data.DataLoader(
    transformed_datasets['val'],
    batch_size=32,
    shuffle=True,
    num_workers=8,
    pin_memory=True)

In [ ]:
import torch
import pandas as pd
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import sklearn
import sklearn.metrics as sklm
from torch.autograd import Variable
import numpy as np


def make_pred_multilabel(model, alignment_network):
    """
    Gives predictions for test fold and calculates AUCs using previously trained model

    Args:
        data_transforms: torchvision transforms to preprocess raw images; same as validation transforms
        model: densenet-121 from torchvision previously fine tuned to training data
        PATH_TO_IMAGES: path at which NIH images can be found
    Returns:
        pred_df: dataframe containing individual predictions and ground truth for each test image
        auc_df: dataframe containing aggregate AUCs by train/test tuples
    """


    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    BATCH_SIZE = 32
    NUM_WORKERS = os.cpu_count()
    data_transforms = {
        'train': transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because resize doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'test': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }
    # calc preds in batches of 16, can reduce if your GPU has less RAM
    BATCH_SIZE = 32

    # set model to eval mode; required for proper predictions given use of batchnorm
    model.train(False)

    # create dataloader
    dataset = CreateDataset(test, "test", transform=data_transforms['test'])
    dataloader = torch.utils.data.DataLoader(
        dataset, BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=True)
    size = len(dataset)

    # create empty dfs
    pred_df = pd.DataFrame(columns=["Image Index"])
    true_df = pd.DataFrame(columns=["Image Index"])

    # iterate over dataloader
    for i, data in enumerate(dataloader):

        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        true_labels = labels.cpu().data.numpy()
        batch_size = true_labels.shape

        outputs = model(inputs, alignment_network, batch_masks, lung)
        probs = outputs.cpu().data.numpy()

        # get predictions and true values for each item in batch
        for j in range(0, batch_size[0]):
            thisrow = {}
            truerow = {}
            thisrow["Image Index"] = dataset.df.index[BATCH_SIZE * i + j]
            truerow["Image Index"] = dataset.df.index[BATCH_SIZE * i + j]

            # iterate over each entry in prediction vector; each corresponds to
            # individual label
            for k in range(len(dataset.PRED_LABEL)):
                thisrow["prob_" + dataset.PRED_LABEL[k]] = probs[j, k]
                truerow[dataset.PRED_LABEL[k]] = true_labels[j, k]

            pred_df = pred_df.append(thisrow, ignore_index=True)
            true_df = true_df.append(truerow, ignore_index=True)

#         if(i % 10 == 0):
#             print(str(i * BATCH_SIZE))

    auc_df = pd.DataFrame(columns=["label", "auc"])

    # calc AUCs
    for column in true_df:
        if column not in [
            'Atelectasis',
            'Cardiomegaly',
            'Effusion',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pneumonia',
            'Pneumothorax']:
                    continue
        actual = true_df[column]
        pred = pred_df["prob_" + column]
        thisrow = {}
        thisrow['label'] = column
        thisrow['auc'] = np.nan
#         try:
#             thisrow['auc'] = sklm.roc_auc_score(
#                 actual.as_matrix().astype(int), pred.as_matrix())
        thisrow['auc'] = sklm.roc_auc_score(actual.astype(int), pred)
#         except BaseException:
#             print("can't calculate auc for " + str(column))
        auc_df = auc_df.append(thisrow, ignore_index=True)

    pred_df.to_csv("results/preds.csv", index=False)
    auc_df.to_csv("results/aucs.csv", index=False)
    return pred_df, auc_df, true_df, actual, pred

# VICReg

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import torch
import torch.nn as nn


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ["downsample"]

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
        last_activation="relu",
    ):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ["downsample"]

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
        last_activation="relu",
    ):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

        if last_activation == "relu":
            self.last_activation = nn.ReLU(inplace=True)
        elif last_activation == "none":
            self.last_activation = nn.Identity()
        elif last_activation == "sigmoid":
            self.last_activation = nn.Sigmoid()

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.last_activation(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block,
        layers,
        num_channels=3,
        zero_init_residual=False,
        groups=1,
        widen=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
        last_activation="relu",
    ):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        # self._last_activation = last_activation

        self.padding = nn.ConstantPad2d(1, 0.0)

        self.inplanes = width_per_group * widen
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                "or a 3-element tuple, got {}".format(replace_stride_with_dilation)
            )
        self.groups = groups
        self.base_width = width_per_group

        # change padding 3 -> 2 compared to original torchvision code because added a padding layer
        num_out_filters = width_per_group * widen
        self.conv1 = nn.Conv2d(
            num_channels,
            num_out_filters,
            kernel_size=7,
            stride=2,
            padding=2,
            bias=False,
        )
        self.bn1 = norm_layer(num_out_filters)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, num_out_filters, layers[0])
        num_out_filters *= 2
        self.layer2 = self._make_layer(
            block,
            num_out_filters,
            layers[1],
            stride=2,
            dilate=replace_stride_with_dilation[0],
        )
        num_out_filters *= 2
        self.layer3 = self._make_layer(
            block,
            num_out_filters,
            layers[2],
            stride=2,
            dilate=replace_stride_with_dilation[1],
        )
        num_out_filters *= 2
        self.layer4 = self._make_layer(
            block,
            num_out_filters,
            layers[3],
            stride=2,
            dilate=replace_stride_with_dilation[2],
            last_activation=last_activation,
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(
        self, block, planes, blocks, stride=1, dilate=False, last_activation="relu"
    ):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes,
                planes,
                stride,
                downsample,
                self.groups,
                self.base_width,
                previous_dilation,
                norm_layer,
                last_activation=(last_activation if blocks == 1 else "relu"),
            )
        )
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                    last_activation=(last_activation if i == blocks - 1 else "relu"),
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.padding(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        return x


def resnet34(**kwargs):
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs), 512


def resnet50(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], **kwargs), 2048


def resnet101(**kwargs):
    return ResNet(Bottleneck, [3, 4, 23, 3], **kwargs), 2048


def resnet50x2(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], widen=2, **kwargs), 4096


def resnet50x4(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], widen=4, **kwargs), 8192


def resnet50x5(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], widen=5, **kwargs), 10240


def resnet200x2(**kwargs):
    return ResNet(Bottleneck, [3, 24, 36, 3], widen=2, **kwargs), 4096

In [ ]:
import torch
import torchvision
from torch import nn

## The projection head is the same as the Barlow Twins one
from lightly.loss import VICRegLoss

## The projection head is the same as the Barlow Twins one
from lightly.loss.vicreg_loss import VICRegLoss
from lightly.models.modules import BarlowTwinsProjectionHead
# from lightly.transforms.vicreg_transform import VICRegTransform


class VICReg(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = BarlowTwinsProjectionHead(2048, 2048, 2048)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

In [ ]:
# backbone = resnet50()
# backbone = backbone[0]
# checkpoint = torch.load('/content/drive/MyDrive/vicreg/resnet50.pth')
# backbone.load_state_dict(checkpoint)

resnet = torchvision.models.resnet50()
backbone = nn.Sequential(*list(resnet.children())[:-1])
ssl_model = VICReg(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
ssl_model.to(device)
print('Model initiated')

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/final_random_vicreg/data_aug_vicreg_chestxray_20_withprojectionhead.pth')
ssl_model.load_state_dict(checkpoint)
backbone = ssl_model.backbone
backbone = nn.Sequential(*list(backbone.children()))
num_ftrs = 2048
backbone.fc = nn.Sequential(
    nn.Flatten(), nn.Linear(num_ftrs, 8), nn.Sigmoid())
backbone.to(device)
print('Model loaded')

In [ ]:
masks = np.load('/content/drive/MyDrive/thesis_models/final_fixed_aligned_layercam_normalized_masks.npy', allow_pickle=True)
for i in range(1, 8):
    masks[i] = masks[i].squeeze(0)
for i in range(len(masks)):
    masks[i] = masks[i].to(torch.float32)

# def ptp(t, axis):
#   return t.max(axis).values - t.min(axis).values

# for i in range(len(masks)):
#   mean, std, var = torch.mean(masks[i]), torch.std(masks[i]), torch.var(masks[i])
#   min = torch.min
#   masks[i]  = (masks[i]-mean)/std
# print('-------')
# for i in masks:
#   mean, std, var = torch.mean(i), torch.std(i), torch.var(i)
#   print(mean, std, var)

# temp = []
# for i in range(7):
#     temp1 = [0., 0., 0., 0., 0., 0., 0.]
#     temp.append(temp1)
# masks[6] = torch.Tensor(temp)

l = []
for i in masks:
    l.append(i)
masks = torch.stack(l)

batch_masks = []
for i in range(32):
    batch_masks.append(masks)

batch_masks = torch.stack(batch_masks).cuda()
batch_masks = batch_masks.to(torch.float32)

In [ ]:
lung = np.load('/content/drive/MyDrive/56_lung_mask.npy', allow_pickle=True)

lung = lung[0]
lung = lung.to(torch.float32)

l = []
for i in range(256):
    l.append(lung)
lung = torch.stack(l).cuda()
lung = lung.to(torch.float32)

In [ ]:
centre_lung = np.load('/content/drive/MyDrive/56_center_lung_mask.npy', allow_pickle=True)

centre_lung = centre_lung[0]
centre_lung = centre_lung.to(torch.float32)

l = []
for i in range(256):
    l.append(centre_lung)
centre_lung = torch.stack(l).cuda()
centre_lung = centre_lung.to(torch.float32)

In [ ]:
from collections import namedtuple
import os
import torch
from torchvision import models
import torch.nn.functional as F
torch.manual_seed(98)

class AlignmentNetwork(torch.nn.Module):
    def __init__(self):
        super(AlignmentNetwork, self).__init__()

        resnet18 = models.resnet18()
        num_ftrs = resnet18.fc.in_features
        resnet18.fc = nn.Linear(num_ftrs, 6)

        self.model  = resnet18


    def forward(self, x):
        theta = self.model(x)
        theta = theta.view(-1, 2, 3).squeeze()

        self.grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, self.grid)
#         m = nn.AvgPool2d(16, stride=16)
#         x = m(x)
#         x = F.interpolate(x, (224,224), mode='bilinear')
        return x

In [ ]:
# the vgg16 is borrowed from the pytorch tutorial
# the resent is used to reimplemented and the comparision experiments
from collections import namedtuple
import os
import torch
from torchvision import models

class ResNet50(torch.nn.Module):
    def __init__(self, backbone):
        super(ResNet50, self).__init__()

        # resnet50_pretrained = models.resnet50()
        # num_ftrs = resnet50_pretrained.fc.in_features
        # resnet50_pretrained.fc = nn.Sequential(
        #     nn.Linear(num_ftrs, 8), nn.Sigmoid())

        resnet50_pretrained = backbone

        # resnet50_pretrained.load_state_dict(torch.load('/kaggle/input/models-for-localisation-testing/resnet50_5_chestxray8.pth'))

        self.model  = list(resnet50_pretrained.children())

        self.noslice1 = self.model[1]
        self.noslice2 = self.model[2]
        self.noslice3 = self.model[3]
        self.noslice4 = self.model[4]

        self.slice1 = self.model[5]
        self.slice2 = self.model[6]
        self.slice3 = self.model[7]
        self.slice4 = self.model[8]

#         self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        self.conv = nn.Sequential(nn.Conv2d(in_channels=2048, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.anothaone = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)), nn.Sigmoid())
        # self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        #                         nn.Flatten(),
        #                         nn.Linear(2048, 8),
        #                         nn.Sigmoid())

    def forward(self, x, alignment_network, batch_masks, lung_mask):
#         x = x.to(device)
        o = x
#         for i in range(4):
#             o = self.model[i](o)

        # alignment_network.eval()
        # o = alignment_network(o)
        # aligned = o

        o = self.noslice1(o)
        o = self.noslice2(o)
        o = self.noslice3(o)
        o = self.noslice4(o)

        # o = (o * lung_mask) + o

#         anchor = cv.resize()
#         o = o*anchor
        o = self.slice1(o)
        feature1 = o
        # o = (o * lung_mask) + o
        o = self.slice2(o)
        feature2 = o
        o = self.slice3(o)
        feature3 = o
        o = self.slice4(o)
        feature4 = o
        o = self.conv(o)
        final = o
        if(o.shape[0] == 32):
          o = (o * batch_masks) + o
        else:
          temp = o.shape[0]
          o = (o * batch_masks[:temp]) + o
        o = self.anothaone(o)
        final2 = o
        o = self.fc(o)
        final3 = o

        return final3

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

vicreg_model = ResNet50(backbone)
vicreg_model = vicreg_model.to(device)
vicreg_model = vicreg_model.type(torch.cuda.FloatTensor)
vicreg_model.load_state_dict(torch.load('/content/drive/MyDrive/thesis_models/dataaug_vicreg_noalign_5.pth'))

# vicreg_model = ResNet50()
# vicreg_model.load_state_dict(torch.load('/content/drive/MyDrive/final_random_init/random_vicreg_oldalign_5.pth'))
# vicreg_model = vicreg_model.to(device)
# vicreg_model = vicreg_model.type(torch.cuda.FloatTensor)

In [ ]:
vicreg_model

In [ ]:
##### https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723/21
# https://www.fast.ai/posts/2018-07-02-adam-weight-decay.html
# https://stackoverflow.com/questions/44452571/what-is-the-proper-way-to-weight-decay-for-adam-optimizer

use_gpu = torch.cuda.is_available()
gpu_count = torch.cuda.device_count()
print("Available GPU count:" + str(gpu_count))


def checkpoint(model, best_loss, epoch, LR):
    """
    Saves checkpoint of torchvision model during training.
    Args:
        model: torchvision model to be saved
        best_loss: best val loss achieved so far in training
        epoch: current epoch of training
        LR: current learning rate in training
    Returns:
        None
    """

    print('saving')
    state = {
        'model': model,
        'best_loss': best_loss,
        'epoch': epoch,
        'rng_state': torch.get_rng_state(),
        'LR': LR
    }

    torch.save(state, 'results/checkpoint')

def get_loss(output, target, index, device):
    import torch.nn.functional as F

    num_classes = [1, 1, 1, 1, 1, 1, 1, 1]
    pos_weight = [1, 1, 1, 1, 1, 1, 1, 1]
    batch_weight = True

    for num_class in num_classes:
        assert num_class == 1
    target = target[:, index].view(-1)
    output = torch.transpose(output, 0, 1)
    pos_weight = torch.from_numpy(
        np.array(pos_weight,
                 dtype=np.float32)).to(device).type_as(target)

    if batch_weight:
        if target.sum() == 0:
            loss = torch.tensor(0., requires_grad=True).to(device)
        else:
            weight = (target.size()[0] - target.sum()) / target.sum()
            loss = F.binary_cross_entropy_with_logits(
                output[index].view(-1), target, pos_weight=weight)
    else:
        loss = F.binary_cross_entropy_with_logits(
            output[index].view(-1), target, pos_weight=pos_weight[index])

    label = torch.sigmoid(output[index].view(-1)).ge(0.5).float()
    acc = (target == label).float().sum() / len(label)

    return (loss, acc)


def train_model(
        model,
        criterion,
        optimizer,
        LR,
        num_epochs,
        dataloaders,
        dataset_sizes,
        weight_decay):
    """
    Fine tunes torchvision model to NIH CXR data.
    Args:
        model: torchvision model to be finetuned (densenet-121 in this case)
        criterion: loss criterion (binary cross entropy loss, BCELoss)
        optimizer: optimizer to use in training (SGD)
        LR: learning rate
        num_epochs: continue training up to this many epochs
        dataloaders: pytorch train and val dataloaders
        dataset_sizes: length of train and val datasets
        weight_decay: weight decay parameter we use in SGD with momentum
    Returns:
        model: trained torchvision model
        best_epoch: epoch on which best model val loss was obtained
    """
    since = time.time()

    start_epoch = 0
    best_loss = 999999
    best_epoch = -1
    last_train_loss = -1

    auc_df_temp = 0
    num_tasks = 8

    device = "cuda" if torch.cuda.is_available() else "cpu"

    alignment_network = AlignmentNetwork()
    alignment_network.load_state_dict(torch.load('/content/drive/MyDrive/2feat_no_crop_fixed_pt_alignment_network_5.pth'))
    alignment_network = alignment_network.to(device)
    alignment_network.eval()

    # iterate over epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # set model to train or eval mode based on whether we are in train or
        # val; necessary to get correct predictions given batchnorm
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0

            i = 0
            total_done = 0
            # iterate over all data in train/val dataloader:
            for data in dataloaders[phase]:
                i += 1
                loss_sum = np.zeros(num_tasks)
                acc_sum = np.zeros(num_tasks)
                loss = 0
                inputs, labels = data
                batch_size = inputs.shape[0]
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda()).float()
                outputs = model(inputs, alignment_network, batch_masks, lung)

                for t in range(num_tasks):
                    loss_t, acc_t = get_loss(outputs, labels, t, device)
                    loss += loss_t
                    loss_sum[t] += loss_t.item()
                    acc_sum[t] += acc_t.item()
                # calculate gradient and update parameters in train phase
                optimizer.zero_grad()
                # loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data * batch_size

            epoch_loss = running_loss / dataset_sizes[phase]

            if phase == 'train':
                last_train_loss = epoch_loss

            print(phase + ' epoch {}:loss {:.4f} with data size {}'.format(
                epoch, epoch_loss, dataset_sizes[phase]))

            if (epoch == (num_epochs - 1)):
                torch.save(model.state_dict(), f'/content/drive/MyDrive/thesis_models/dataaug_vicreg_layerdmask_noalign_{epoch+6}.pth')

            # decay learning rate if no val loss improvement in this epoch

#             if phase == 'val' and epoch_loss > best_loss:
#                 print("decay loss from " + str(LR) + " to " +
#                       str(LR / 10) + " as not seeing improvement in val loss")
#                 LR = LR / 10
#                 # create new optimizer with lower learning rate
# #                 optimizer = optim.SGD(
# #                     filter(
# #                         lambda p: p.requires_grad,
# #                         model.parameters()),
# #                     lr=LR,
# #                     momentum=0.9,
# #                     weight_decay=weight_decay)
#                 optimizer = optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.999))
#                 print("created new optimizer with LR " + str(LR))

            # checkpoint model if has best val loss yet
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                checkpoint(model, best_loss, epoch, LR)

            # log training and validation loss over each epoch
            if phase == 'val':
                with open("results/log_train", 'a') as logfile:
                    logwriter = csv.writer(logfile, delimiter=',')
                    if(epoch == 1):
                        logwriter.writerow(["epoch", "train_loss", "val_loss"])
                    logwriter.writerow([epoch, last_train_loss, epoch_loss])

        total_done += batch_size
        if(total_done % (100 * batch_size) == 0):
            print("completed " + str(total_done) + " so far in epoch")

        # break if no val loss improvement in 3 epochs
#         if ((epoch - best_epoch) >= 3):
#             print("no improvement in 3 epochs, break")
#             break

        pred_df, auc_df, true_df, actual, pred = make_pred_multilabel(model, alignment_network)
        if (auc_df_temp > auc_df['auc'].mean()):
            auc_df_temp = auc_df['auc'].mean()
            print('No increase in AUC. AUC: ', auc_df['auc'].mean())
        if (auc_df_temp < auc_df['auc'].mean()):
            auc_df_temp = auc_df['auc'].mean()
            print('AUC: ', auc_df['auc'].mean())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights to return
    checkpoint_best = torch.load('results/checkpoint')
    model = checkpoint_best['model']

    return model, best_epoch


def train_cnn(LR, WEIGHT_DECAY, model):
    """
    Train torchvision model to NIH data given high level hyperparameters.
    Args:
        PATH_TO_IMAGES: path to NIH images
        LR: learning rate
        WEIGHT_DECAY: weight decay parameter for SGD
    Returns:
        preds: torchvision model predictions on test fold with ground truth for comparison
        aucs: AUCs for each train,test tuple
    """
    NUM_EPOCHS = 5
    BATCH_SIZE = 32

    try:
        rmtree('results/')
    except BaseException:
        pass  # directory doesn't yet exist, no need to clear it
    os.makedirs("results/")

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    N_LABELS = 8  # we are predicting 14 labels
    data_transforms = {
        'train': transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because resize doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'test': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

    transformed_datasets = {}
    transformed_datasets['train'] = CreateDataset(train, 'train', transform=data_transforms['train'])
    transformed_datasets['val'] = CreateDataset(train, 'val', transform=data_transforms['val'])

    dataloaders = {}
    dataloaders['train'] = torch.utils.data.DataLoader(
        transformed_datasets['train'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)
    dataloaders['val'] = torch.utils.data.DataLoader(
        transformed_datasets['val'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)

    # please do not attempt to train without GPU as will take excessively long
    if not use_gpu:
        raise ValueError("Error, requires GPU")

    # model = models.resnet50(weights="IMAGENET1K_V2")
    # num_ftrs = model.fc.in_features
    # model.fc = nn.Sequential(
    #     nn.Linear(num_ftrs, N_LABELS), nn.Sigmoid())

    # put model on GPU
    model = model.cuda()

#     checkpoint = torch.load('/kaggle/input/chestx-ray/resnet50_5_chestxray8.pth')
#     model.load_state_dict(checkpoint)

    # define criterion, optimizer for training
    criterion = nn.BCELoss()
    # criterion = FocalLoss()

#     optimizer = optim.Adam(
#         filter(
#             lambda p: p.requires_grad,
#             model.parameters()),
#         lr=LR,
#         weight_decay=WEIGHT_DECAY)
    optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
#     optimizer = optim.SGD(
#         filter(
#             lambda p: p.requires_grad,
#             model.parameters()),
#         lr=LR,
#         momentum=0.9,
#         weight_decay=WEIGHT_DECAY)
    dataset_sizes = {x: len(transformed_datasets[x]) for x in ['train', 'val']}

    # train model
    model, best_epoch = train_model(model, criterion, optimizer, LR, num_epochs=NUM_EPOCHS,
                                    dataloaders=dataloaders, dataset_sizes=dataset_sizes, weight_decay=WEIGHT_DECAY)
    print('Finished training')

    # get preds and AUCs on test fold

    return model, best_epoch

In [ ]:
import os

WEIGHT_DECAY = 0.01
LEARNING_RATE = 0.0001
model, best_epoch = train_cnn(LEARNING_RATE, WEIGHT_DECAY, vicreg_model)

# dataaug_vicreg_layerdmask_noalign_10

# SimSIAM

In [ ]:
import torch
import torchvision
from torch import nn

from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
# from lightly.transforms import SimSiamTransform


class SimSiam_Train(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SimSiamProjectionHead(2048, 512, 128)
        self.prediction_head = SimSiamPredictionHead(128, 64, 128)

    def forward(self, x):
        f = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(f)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import torch
import torch.nn as nn


class SimSiam(nn.Module):
    """
    Build a SimSiam model.
    """
    def __init__(self, base_encoder, dim=2048, pred_dim=512):
        """
        dim: feature dimension (default: 2048)
        pred_dim: hidden dimension of the predictor (default: 512)
        """
        super(SimSiam, self).__init__()

        # create the encoder
        # num_classes is the output fc dimension, zero-initialize last BNs
        self.encoder = base_encoder(num_classes=dim, zero_init_residual=True)

        # build a 3-layer projector
        prev_dim = self.encoder.fc.weight.shape[1]
        self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        self.encoder.fc,
                                        nn.BatchNorm1d(dim, affine=False)) # output layer
        self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        # build a 2-layer predictor
        self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, dim)) # output layer

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """

        # compute features for one view
        z1 = self.encoder(x1) # NxC
        z2 = self.encoder(x2) # NxC

        p1 = self.predictor(z1) # NxC
        p2 = self.predictor(z2) # NxC

        return p1, p2, z1.detach(), z2.detach()

In [ ]:
model = SimSiam(
    models.__dict__['resnet50'],
    2048, 512)

model = nn.DataParallel(model)

# ckpt = torch.load('/kaggle/input/chestx-ray/fastsiam_2_resnet50_backbonemodel.pth')
# backbone.load_state_dict(ckpt['resnet50_parameters'])

# checkpoint = torch.load('/content/drive/MyDrive/simsiam/checkpoint_0099.pth.tar')
# model.load_state_dict(checkpoint['state_dict'])

backbone = nn.Sequential(*list(model.module.children())[:-1])
backbone = nn.Sequential(*list(backbone[0].children())[:-1])

ssl_model = SimSiam_Train(backbone)

# checkpoint = torch.load('/content/pretrained_simsiam_28_withprojectionhead.pth')
# model.load_state_dict(checkpoint)

device = "cuda" if torch.cuda.is_available() else "cpu"
ssl_model = ssl_model.to(device)
print('Model ready for training')

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/simsiam/simsiam_chestxray_20_withprojectionhead.pth')
ssl_model.load_state_dict(checkpoint)
backbone = ssl_model.backbone
backbone = nn.Sequential(*list(backbone.children()))
num_ftrs = 2048
backbone.fc = nn.Sequential(
    nn.Flatten(), nn.Linear(num_ftrs, 8), nn.Sigmoid())
backbone.to(device)
print('Model loaded')

In [ ]:
# the vgg16 is borrowed from the pytorch tutorial
# the resent is used to reimplemented and the comparision experiments
from collections import namedtuple
import os
import torch
from torchvision import models

class ResNet50(torch.nn.Module):
    def __init__(self, backbone):
        super(ResNet50, self).__init__()

        resnet50_pretrained = backbone

        # resnet50_pretrained.load_state_dict(torch.load('/kaggle/input/models-for-localisation-testing/resnet50_5_chestxray8.pth'))

        self.model  = list(resnet50_pretrained.children())

        self.noslice1 = self.model[0]
        self.noslice2 = self.model[1]
        self.noslice3 = self.model[2]
        self.noslice4 = self.model[3]

        self.slice1 = self.model[4]
        self.slice2 = self.model[5]
        self.slice3 = self.model[6]
        self.slice4 = self.model[7]

#         self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        self.conv = nn.Sequential(nn.Conv2d(in_channels=2048, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.anothaone = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)), nn.Sigmoid())
        # self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        #                         nn.Flatten(),
        #                         nn.Linear(2048, 8),
        #                         nn.Sigmoid())

    def forward(self, x):
#         x = x.to(device)
        o = x
#         for i in range(4):
#             o = self.model[i](o)

        o = self.noslice1(o)
        o = self.noslice2(o)
        o = self.noslice3(o)
        o = self.noslice4(o)

#         anchor = cv.resize()
#         o = o*anchor
        o = self.slice1(o)
        feature1 = o
        o = self.slice2(o)
        feature2 = o
        o = self.slice3(o)
        feature3 = o
        o = self.slice4(o)
        feature4 = o
        o = self.conv(o)
        final = o
        # if(o.shape[0] == 32):
        #   o = (o * batch_masks) + o
        # else:
        #   temp = o.shape[0]
        #   o = (o * batch_masks[:temp]) + o
        o = self.anothaone(o)
        final2 = o
        o = self.fc(o)
        final3 = o

        return final3

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

simsiam_model = ResNet50(backbone)
simsiam_model = simsiam_model.to(device)
simsiam_model = simsiam_model.type(torch.cuda.FloatTensor)

In [ ]:
##### https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723/21
# https://www.fast.ai/posts/2018-07-02-adam-weight-decay.html
# https://stackoverflow.com/questions/44452571/what-is-the-proper-way-to-weight-decay-for-adam-optimizer

use_gpu = torch.cuda.is_available()
gpu_count = torch.cuda.device_count()
print("Available GPU count:" + str(gpu_count))


def checkpoint(model, best_loss, epoch, LR):
    """
    Saves checkpoint of torchvision model during training.
    Args:
        model: torchvision model to be saved
        best_loss: best val loss achieved so far in training
        epoch: current epoch of training
        LR: current learning rate in training
    Returns:
        None
    """

    print('saving')
    state = {
        'model': model,
        'best_loss': best_loss,
        'epoch': epoch,
        'rng_state': torch.get_rng_state(),
        'LR': LR
    }

    torch.save(state, 'results/checkpoint')

def get_loss(output, target, index, device):
    import torch.nn.functional as F

    num_classes = [1, 1, 1, 1, 1, 1, 1, 1]
    pos_weight = [1, 1, 1, 1, 1, 1, 1, 1]
    batch_weight = True

    for num_class in num_classes:
        assert num_class == 1
    target = target[:, index].view(-1)
    output = torch.transpose(output, 0, 1)
    pos_weight = torch.from_numpy(
        np.array(pos_weight,
                 dtype=np.float32)).to(device).type_as(target)

    if batch_weight:
        if target.sum() == 0:
            loss = torch.tensor(0., requires_grad=True).to(device)
        else:
            weight = (target.size()[0] - target.sum()) / target.sum()
            loss = F.binary_cross_entropy_with_logits(
                output[index].view(-1), target, pos_weight=weight)
    else:
        loss = F.binary_cross_entropy_with_logits(
            output[index].view(-1), target, pos_weight=pos_weight[index])

    label = torch.sigmoid(output[index].view(-1)).ge(0.5).float()
    acc = (target == label).float().sum() / len(label)

    return (loss, acc)


def train_model(
        model,
        criterion,
        optimizer,
        LR,
        num_epochs,
        dataloaders,
        dataset_sizes,
        weight_decay):
    """
    Fine tunes torchvision model to NIH CXR data.
    Args:
        model: torchvision model to be finetuned (densenet-121 in this case)
        criterion: loss criterion (binary cross entropy loss, BCELoss)
        optimizer: optimizer to use in training (SGD)
        LR: learning rate
        num_epochs: continue training up to this many epochs
        dataloaders: pytorch train and val dataloaders
        dataset_sizes: length of train and val datasets
        weight_decay: weight decay parameter we use in SGD with momentum
    Returns:
        model: trained torchvision model
        best_epoch: epoch on which best model val loss was obtained
    """
    since = time.time()

    start_epoch = 0
    best_loss = 999999
    best_epoch = -1
    last_train_loss = -1

    auc_df_temp = 0
    num_tasks = 8

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # iterate over epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # set model to train or eval mode based on whether we are in train or
        # val; necessary to get correct predictions given batchnorm
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0

            i = 0
            total_done = 0
            # iterate over all data in train/val dataloader:
            for data in dataloaders[phase]:
                i += 1
                loss_sum = np.zeros(num_tasks)
                acc_sum = np.zeros(num_tasks)
                loss = 0
                inputs, labels = data
                batch_size = inputs.shape[0]
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda()).float()
                outputs = model(inputs)

                for t in range(num_tasks):
                    loss_t, acc_t = get_loss(outputs, labels, t, device)
                    loss += loss_t
                    loss_sum[t] += loss_t.item()
                    acc_sum[t] += acc_t.item()
                # calculate gradient and update parameters in train phase
                optimizer.zero_grad()
                # loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data * batch_size

            epoch_loss = running_loss / dataset_sizes[phase]

            if phase == 'train':
                last_train_loss = epoch_loss

            print(phase + ' epoch {}:loss {:.4f} with data size {}'.format(
                epoch, epoch_loss, dataset_sizes[phase]))

            torch.save(model.state_dict(), f'/content/drive/MyDrive/final_ssl/simsiam_resnet50_{epoch+1}.pth')

            # decay learning rate if no val loss improvement in this epoch

#             if phase == 'val' and epoch_loss > best_loss:
#                 print("decay loss from " + str(LR) + " to " +
#                       str(LR / 10) + " as not seeing improvement in val loss")
#                 LR = LR / 10
#                 # create new optimizer with lower learning rate
# #                 optimizer = optim.SGD(
# #                     filter(
# #                         lambda p: p.requires_grad,
# #                         model.parameters()),
# #                     lr=LR,
# #                     momentum=0.9,
# #                     weight_decay=weight_decay)
#                 optimizer = optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.999))
#                 print("created new optimizer with LR " + str(LR))

            # checkpoint model if has best val loss yet
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                checkpoint(model, best_loss, epoch, LR)

            # log training and validation loss over each epoch
            if phase == 'val':
                with open("results/log_train", 'a') as logfile:
                    logwriter = csv.writer(logfile, delimiter=',')
                    if(epoch == 1):
                        logwriter.writerow(["epoch", "train_loss", "val_loss"])
                    logwriter.writerow([epoch, last_train_loss, epoch_loss])

        total_done += batch_size
        if(total_done % (100 * batch_size) == 0):
            print("completed " + str(total_done) + " so far in epoch")

        # break if no val loss improvement in 3 epochs
#         if ((epoch - best_epoch) >= 3):
#             print("no improvement in 3 epochs, break")
#             break

        pred_df, auc_df, true_df, actual, pred = make_pred_multilabel(model)
        if (auc_df_temp > auc_df['auc'].mean()):
            auc_df_temp = auc_df['auc'].mean()
            print('No increase in AUC. AUC: ', auc_df['auc'].mean())
        if (auc_df_temp < auc_df['auc'].mean()):
            auc_df_temp = auc_df['auc'].mean()
            print('AUC: ', auc_df['auc'].mean())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights to return
    checkpoint_best = torch.load('results/checkpoint')
    model = checkpoint_best['model']

    return model, best_epoch


def train_cnn(LR, WEIGHT_DECAY, model):
    """
    Train torchvision model to NIH data given high level hyperparameters.
    Args:
        PATH_TO_IMAGES: path to NIH images
        LR: learning rate
        WEIGHT_DECAY: weight decay parameter for SGD
    Returns:
        preds: torchvision model predictions on test fold with ground truth for comparison
        aucs: AUCs for each train,test tuple
    """
    NUM_EPOCHS = 7
    BATCH_SIZE = 32

    try:
        rmtree('results/')
    except BaseException:
        pass  # directory doesn't yet exist, no need to clear it
    os.makedirs("results/")

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    N_LABELS = 8  # we are predicting 14 labels
    data_transforms = {
        'train': transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because resize doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'test': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

    transformed_datasets = {}
    transformed_datasets['train'] = CreateDataset(train, 'train', transform=data_transforms['train'])
    transformed_datasets['val'] = CreateDataset(train, 'val', transform=data_transforms['val'])

    dataloaders = {}
    dataloaders['train'] = torch.utils.data.DataLoader(
        transformed_datasets['train'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)
    dataloaders['val'] = torch.utils.data.DataLoader(
        transformed_datasets['val'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)

    # please do not attempt to train without GPU as will take excessively long
    if not use_gpu:
        raise ValueError("Error, requires GPU")

    # model = models.resnet50(weights="IMAGENET1K_V2")
    # num_ftrs = model.fc.in_features
    # model.fc = nn.Sequential(
    #     nn.Linear(num_ftrs, N_LABELS), nn.Sigmoid())

    # put model on GPU
    model = model.cuda()

#     checkpoint = torch.load('/kaggle/input/chestx-ray/resnet50_5_chestxray8.pth')
#     model.load_state_dict(checkpoint)

    # define criterion, optimizer for training
    criterion = nn.BCELoss()
    # criterion = FocalLoss()

#     optimizer = optim.Adam(
#         filter(
#             lambda p: p.requires_grad,
#             model.parameters()),
#         lr=LR,
#         weight_decay=WEIGHT_DECAY)
    optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
#     optimizer = optim.SGD(
#         filter(
#             lambda p: p.requires_grad,
#             model.parameters()),
#         lr=LR,
#         momentum=0.9,
#         weight_decay=WEIGHT_DECAY)
    dataset_sizes = {x: len(transformed_datasets[x]) for x in ['train', 'val']}

    # train model
    model, best_epoch = train_model(model, criterion, optimizer, LR, num_epochs=NUM_EPOCHS,
                                    dataloaders=dataloaders, dataset_sizes=dataset_sizes, weight_decay=WEIGHT_DECAY)
    print('Finished training')

    # get preds and AUCs on test fold

    return model, best_epoch

In [ ]:
import os

WEIGHT_DECAY = 0.01
LEARNING_RATE = 0.0001
model, best_epoch = train_cnn(LEARNING_RATE, WEIGHT_DECAY, simsiam_model)

# Barlow

In [ ]:
class BarlowTwins(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = BarlowTwinsProjectionHead(2048, 2048, 2048)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

In [ ]:
backbone = torchvision.models.resnet50()
# backbone = nn.DataParallel(backbone)
# checkpoint = torch.load('/content/drive/MyDrive/barlow/resnet50.pth')
# backbone.load_state_dict(checkpoint, strict=False)
backbone = nn.Sequential(*list(backbone.children())[:-1])

In [ ]:
ssl_model = BarlowTwins(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
ssl_model.to(device)

checkpoint = torch.load('/content/drive/MyDrive/barlow/barlow_chestxray_20_withprojectionhead.pth')
ssl_model.load_state_dict(checkpoint)

In [ ]:
backbone = ssl_model.backbone
backbone = nn.Sequential(*list(backbone.children()))
num_ftrs = 2048
backbone.fc = nn.Sequential(
    nn.Flatten(), nn.Linear(num_ftrs, 8), nn.Sigmoid())
backbone.to(device)
print('Model loaded')

In [ ]:
# the vgg16 is borrowed from the pytorch tutorial
# the resent is used to reimplemented and the comparision experiments
from collections import namedtuple
import os
import torch
from torchvision import models

class ResNet50(torch.nn.Module):
    def __init__(self, backbone):
        super(ResNet50, self).__init__()

        resnet50_pretrained = backbone

        # resnet50_pretrained.load_state_dict(torch.load('/kaggle/input/models-for-localisation-testing/resnet50_5_chestxray8.pth'))

        self.model  = list(resnet50_pretrained.children())

        self.noslice1 = self.model[0]
        self.noslice2 = self.model[1]
        self.noslice3 = self.model[2]
        self.noslice4 = self.model[3]

        self.slice1 = self.model[4]
        self.slice2 = self.model[5]
        self.slice3 = self.model[6]
        self.slice4 = self.model[7]

#         self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        self.conv = nn.Sequential(nn.Conv2d(in_channels=2048, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.anothaone = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)), nn.Sigmoid())
        # self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        #                         nn.Flatten(),
        #                         nn.Linear(2048, 8),
        #                         nn.Sigmoid())

    def forward(self, x):
#         x = x.to(device)
        o = x
#         for i in range(4):
#             o = self.model[i](o)

        o = self.noslice1(o)
        o = self.noslice2(o)
        o = self.noslice3(o)
        o = self.noslice4(o)

#         anchor = cv.resize()
#         o = o*anchor
        o = self.slice1(o)
        feature1 = o
        o = self.slice2(o)
        feature2 = o
        o = self.slice3(o)
        feature3 = o
        o = self.slice4(o)
        feature4 = o
        o = self.conv(o)
        final = o
        # if(o.shape[0] == 32):
        #   o = (o * batch_masks) + o
        # else:
        #   temp = o.shape[0]
        #   o = (o * batch_masks[:temp]) + o
        o = self.anothaone(o)
        final2 = o
        o = self.fc(o)
        final3 = o

        return final3

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

barlow_model = ResNet50(backbone)
barlow_model = barlow_model.to(device)
barlow_model = barlow_model.type(torch.cuda.FloatTensor)

In [ ]:
##### https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723/21
# https://www.fast.ai/posts/2018-07-02-adam-weight-decay.html
# https://stackoverflow.com/questions/44452571/what-is-the-proper-way-to-weight-decay-for-adam-optimizer

use_gpu = torch.cuda.is_available()
gpu_count = torch.cuda.device_count()
print("Available GPU count:" + str(gpu_count))


def checkpoint(model, best_loss, epoch, LR):
    """
    Saves checkpoint of torchvision model during training.
    Args:
        model: torchvision model to be saved
        best_loss: best val loss achieved so far in training
        epoch: current epoch of training
        LR: current learning rate in training
    Returns:
        None
    """

    print('saving')
    state = {
        'model': model,
        'best_loss': best_loss,
        'epoch': epoch,
        'rng_state': torch.get_rng_state(),
        'LR': LR
    }

    torch.save(state, 'results/checkpoint')

def get_loss(output, target, index, device):
    import torch.nn.functional as F

    num_classes = [1, 1, 1, 1, 1, 1, 1, 1]
    pos_weight = [1, 1, 1, 1, 1, 1, 1, 1]
    batch_weight = True

    for num_class in num_classes:
        assert num_class == 1
    target = target[:, index].view(-1)
    output = torch.transpose(output, 0, 1)
    pos_weight = torch.from_numpy(
        np.array(pos_weight,
                 dtype=np.float32)).to(device).type_as(target)

    if batch_weight:
        if target.sum() == 0:
            loss = torch.tensor(0., requires_grad=True).to(device)
        else:
            weight = (target.size()[0] - target.sum()) / target.sum()
            loss = F.binary_cross_entropy_with_logits(
                output[index].view(-1), target, pos_weight=weight)
    else:
        loss = F.binary_cross_entropy_with_logits(
            output[index].view(-1), target, pos_weight=pos_weight[index])

    label = torch.sigmoid(output[index].view(-1)).ge(0.5).float()
    acc = (target == label).float().sum() / len(label)

    return (loss, acc)


def train_model(
        model,
        criterion,
        optimizer,
        LR,
        num_epochs,
        dataloaders,
        dataset_sizes,
        weight_decay):
    """
    Fine tunes torchvision model to NIH CXR data.
    Args:
        model: torchvision model to be finetuned (densenet-121 in this case)
        criterion: loss criterion (binary cross entropy loss, BCELoss)
        optimizer: optimizer to use in training (SGD)
        LR: learning rate
        num_epochs: continue training up to this many epochs
        dataloaders: pytorch train and val dataloaders
        dataset_sizes: length of train and val datasets
        weight_decay: weight decay parameter we use in SGD with momentum
    Returns:
        model: trained torchvision model
        best_epoch: epoch on which best model val loss was obtained
    """
    since = time.time()

    start_epoch = 0
    best_loss = 999999
    best_epoch = -1
    last_train_loss = -1

    auc_df_temp = 0
    num_tasks = 8

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # iterate over epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # set model to train or eval mode based on whether we are in train or
        # val; necessary to get correct predictions given batchnorm
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0

            i = 0
            total_done = 0
            # iterate over all data in train/val dataloader:
            for data in dataloaders[phase]:
                i += 1
                loss_sum = np.zeros(num_tasks)
                acc_sum = np.zeros(num_tasks)
                loss = 0
                inputs, labels = data
                batch_size = inputs.shape[0]
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda()).float()
                outputs = model(inputs)

                for t in range(num_tasks):
                    loss_t, acc_t = get_loss(outputs, labels, t, device)
                    loss += loss_t
                    loss_sum[t] += loss_t.item()
                    acc_sum[t] += acc_t.item()
                # calculate gradient and update parameters in train phase
                optimizer.zero_grad()
                # loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data * batch_size

            epoch_loss = running_loss / dataset_sizes[phase]

            if phase == 'train':
                last_train_loss = epoch_loss

            print(phase + ' epoch {}:loss {:.4f} with data size {}'.format(
                epoch, epoch_loss, dataset_sizes[phase]))

            torch.save(model.state_dict(), f'/content/drive/MyDrive/final_ssl/barlow_resnet50_{epoch+1}.pth')

            # decay learning rate if no val loss improvement in this epoch

#             if phase == 'val' and epoch_loss > best_loss:
#                 print("decay loss from " + str(LR) + " to " +
#                       str(LR / 10) + " as not seeing improvement in val loss")
#                 LR = LR / 10
#                 # create new optimizer with lower learning rate
# #                 optimizer = optim.SGD(
# #                     filter(
# #                         lambda p: p.requires_grad,
# #                         model.parameters()),
# #                     lr=LR,
# #                     momentum=0.9,
# #                     weight_decay=weight_decay)
#                 optimizer = optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.999))
#                 print("created new optimizer with LR " + str(LR))

            # checkpoint model if has best val loss yet
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                checkpoint(model, best_loss, epoch, LR)

            # log training and validation loss over each epoch
            if phase == 'val':
                with open("results/log_train", 'a') as logfile:
                    logwriter = csv.writer(logfile, delimiter=',')
                    if(epoch == 1):
                        logwriter.writerow(["epoch", "train_loss", "val_loss"])
                    logwriter.writerow([epoch, last_train_loss, epoch_loss])

        total_done += batch_size
        if(total_done % (100 * batch_size) == 0):
            print("completed " + str(total_done) + " so far in epoch")

        # break if no val loss improvement in 3 epochs
#         if ((epoch - best_epoch) >= 3):
#             print("no improvement in 3 epochs, break")
#             break

        pred_df, auc_df, true_df, actual, pred = make_pred_multilabel(model)
        if (auc_df_temp > auc_df['auc'].mean()):
            auc_df_temp = auc_df['auc'].mean()
            print('No increase in AUC. AUC: ', auc_df['auc'].mean())
        if (auc_df_temp < auc_df['auc'].mean()):
            auc_df_temp = auc_df['auc'].mean()
            print('AUC: ', auc_df['auc'].mean())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights to return
    checkpoint_best = torch.load('results/checkpoint')
    model = checkpoint_best['model']

    return model, best_epoch


def train_cnn(LR, WEIGHT_DECAY, model):
    """
    Train torchvision model to NIH data given high level hyperparameters.
    Args:
        PATH_TO_IMAGES: path to NIH images
        LR: learning rate
        WEIGHT_DECAY: weight decay parameter for SGD
    Returns:
        preds: torchvision model predictions on test fold with ground truth for comparison
        aucs: AUCs for each train,test tuple
    """
    NUM_EPOCHS = 7
    BATCH_SIZE = 32

    try:
        rmtree('results/')
    except BaseException:
        pass  # directory doesn't yet exist, no need to clear it
    os.makedirs("results/")

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    N_LABELS = 8  # we are predicting 14 labels
    data_transforms = {
        'train': transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because resize doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'test': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

    transformed_datasets = {}
    transformed_datasets['train'] = CreateDataset(train, 'train', transform=data_transforms['train'])
    transformed_datasets['val'] = CreateDataset(train, 'val', transform=data_transforms['val'])

    dataloaders = {}
    dataloaders['train'] = torch.utils.data.DataLoader(
        transformed_datasets['train'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)
    dataloaders['val'] = torch.utils.data.DataLoader(
        transformed_datasets['val'],
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)

    # please do not attempt to train without GPU as will take excessively long
    if not use_gpu:
        raise ValueError("Error, requires GPU")

    # model = models.resnet50(weights="IMAGENET1K_V2")
    # num_ftrs = model.fc.in_features
    # model.fc = nn.Sequential(
    #     nn.Linear(num_ftrs, N_LABELS), nn.Sigmoid())

    # put model on GPU
    model = model.cuda()

#     checkpoint = torch.load('/kaggle/input/chestx-ray/resnet50_5_chestxray8.pth')
#     model.load_state_dict(checkpoint)

    # define criterion, optimizer for training
    criterion = nn.BCELoss()
    # criterion = FocalLoss()

#     optimizer = optim.Adam(
#         filter(
#             lambda p: p.requires_grad,
#             model.parameters()),
#         lr=LR,
#         weight_decay=WEIGHT_DECAY)
    optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999))
#     optimizer = optim.SGD(
#         filter(
#             lambda p: p.requires_grad,
#             model.parameters()),
#         lr=LR,
#         momentum=0.9,
#         weight_decay=WEIGHT_DECAY)
    dataset_sizes = {x: len(transformed_datasets[x]) for x in ['train', 'val']}

    # train model
    model, best_epoch = train_model(model, criterion, optimizer, LR, num_epochs=NUM_EPOCHS,
                                    dataloaders=dataloaders, dataset_sizes=dataset_sizes, weight_decay=WEIGHT_DECAY)
    print('Finished training')

    # get preds and AUCs on test fold

    return model, best_epoch

In [ ]:
import os

WEIGHT_DECAY = 0.01
LEARNING_RATE = 0.0001
model, best_epoch = train_cnn(LEARNING_RATE, WEIGHT_DECAY, barlow_model)